<a href='bokeh.pydata.org'><img src="assets/bokeh_logo.svg" alt="Bokeh logo" width="4%;" align="right"/></a>

<a href='http://www.holoviews.org'><img src="assets/header_logo.png" alt="HoloViews logo" width="20%;" align="left"/></a>
<div style="float:right;"><h2>03. Exploration with Containers</h2></div>

In [ ]:
import numpy as np
import holoviews as hv
hv.extension('bokeh')
%opts Curve Area [width=600]

# Declaring elements in a function

Functions allow declaring Elements varying by one or more parameters, which we can explore:

* The function could be loading data from disk
* Querying data from an API
* Generating data from a simulation or mathematical function

For simplicities sake we will declare a simple function generating frequency modulated signal and returns [``Curve``](http://holoviews.org/reference/elements/bokeh/Curve.html) Element:

In [ ]:
def fm_modulation(f_carrier=110, f_mod=110, mod_index=1, length=0.1, sampleRate=3000):
    x = np.arange(0, length, 1.0/sampleRate)
    y = np.sin(2*np.pi*f_carrier*x + mod_index*np.sin(2*np.pi*f_mod*x))
    return hv.Curve((x, y), kdims=['Time'], vdims=['Amplitude'])

The function defines a number of parameters which vary the signal, but let's have a look at what it looks like by default:

In [ ]:
fm_modulation()

## HoloMaps

HoloMaps allow exploring a parameter space with discrete values and are as easy to create as a dictionary comprehension. When declaring a [``HoloMap``](http://holoviews.org/reference/containers/bokeh/HoloMap.html) ensure the length of the key matches the key dimensions:

In [ ]:
carrier_frequencies = [10, 20, 110, 220, 330]
modulation_frequencies = [110, 220, 330]

hmap = hv.HoloMap({(fc, fm): fm_modulation(fc, fm) for fc in carrier_frequencies
                   for fm in modulation_frequencies}, kdims=['fc', 'fm'])
hmap

#### Exercise 1: Change the Element type

* Use the function declared below as a template
* Now change the Element type return by the function to an [``Area``](http://holoviews.org/reference/elements/bokeh/Area.html) or [``Scatter``](http://holoviews.org/reference/elements/bokeh/Area.html)
* Declare lists of ``f_carrier`` and ``mod_index`` values (Don't use too many values here)
* Declare a ``HoloMap`` by iterating over ``f_carrier`` and ``mod_index`` in a dictionary comprehension:

In [ ]:
def fm_modulation2(f_carrier=220, f_mod=110, mod_index=1, length=0.1, sampleRate=3000):
    x = np.arange(0,length, 1.0/sampleRate)
    y = np.sin(2*np.pi*f_carrier*x + mod_index*np.sin(2*np.pi*f_mod*x))

#### Summary

* HoloMaps allow declaring a parameter space
* The default widgets provide a slider for numeric and dropdown menu for non-numeric types.
* HoloMap exports to static HTML files, gifs and videos

## DynamicMap

A [``DynamicMap``]((holoviews.org/reference/containers/bokeh/DynamicMap.html) is very similar to a ``HoloMap`` except that it evaluates the function lazily. This allows exploring much larger parameter spaces (but requires a live server). The key dimensions ``kdims`` must now match the also match the arguments of the function:

In [ ]:
dmap = hv.DynamicMap(fm_modulation, kdims=['f_carrier', 'f_mod', 'mod_index'])
dmap = dmap.redim.range(f_carrier=((10, 110)), f_mod=(10, 110), mod_index=(0.1, 2))
dmap

#### Excercise 2: Declare a DynamicMap

* Reuse your function from Excercise 1
* Declare a DynamicMap with key dimensions, and set ranges using the ``.redim.range`` method.
* (Optional): Use the ``.redim.step`` method and a floating point range to modify the slider step.

## Faceting parameter spaces

### Casting

Instead of widgets we can facet our data in different ways to compare it more directly. Other containers, similar to a ``HoloMap``, such as a [``GridSpace``](http://holoviews.org/reference/elements/bokeh/GridSpace.html), [``NdLayout``](http://holoviews.org/reference/elements/bokeh/NdLayout.html) and [``NdOverlay``](http://holoviews.org/reference/elements/bokeh/NdOverlay.html) let us do that:

In [ ]:
%%opts Curve [width=150]
hv.GridSpace(hmap).opts()

#### Exercise 3: Casting your HoloMap

* Try casting your HoloMap from excercise 1 to an [``NdLayout``](http://holoviews.org/reference/elements/bokeh/NdLayout.html) or [``NdOverlay``](http://holoviews.org/reference/elements/bokeh/NdOverlay.html)

### Faceting with methods

Using the ``.overlay``, ``.grid`` and ``.layout`` methods we can facet multi-dimensional data by a specific dimension:

In [ ]:
hmap.overlay('fm')

These methods, unlike casting, will also work with a ``DynamicMap`` but only if we have defined specific dimension ``values``, which we can do with the ``.redim.values`` method:

In [ ]:
%%opts Curve [width=150]
dmap.redim.values(f_mod=[10, 20, 30], f_carrier=[10, 20, 30]).overlay('f_mod').grid('f_carrier')

#### Exercise 4: Facet a DynamicMap

* Copy your DynamicMap from exercise 2
* Use the redim.values method to declare a list of discrete values for ``f_mod`` and ``f_carrier`` dimensions
* Use the ``.overlay`` and ``.layout`` method to facet the space

## Optional

### Slicing and indexing

HoloMaps and other containers also allow you to easily index or select a specific key, allowing you:

* select a specific key: ``obj[10, 110]``
* select a slice ``obj[10, 200:]``
* select multiple values ``obj[[10, 110], 110]``

In [ ]:
%%opts Curve [width=300]
hmap[10, 110] + hmap[10, 200:].overlay() + hmap[[10, 110], 110].overlay()

You can do the same using the select method:

In [ ]:
(hmap.select(fc=10, fm=110) +
 hmap.select(fc=10, fm=(200, None)).overlay() +
 hmap.select(fc=[10, 110], fm=110).overlay())

## Read more

* Learn more about using HoloMaps and other containers in the [Dimensioned Containers](http://holoviews.org/user_guide/Dimensioned_Containers.html) user guide
* Learn more about working with DynamicMap in the [Live Data](http://holoviews.org/user_guide/Live_Data.html) user guide.